In [4]:
import json
import re
import logging
from typing import Any, Dict, Optional, Union

logger = logging.getLogger(__name__)

def parse_json_safely(text: str) -> Optional[Union[Dict[str, Any], list]]:
    """
    Attempts to parse JSON from the given text using multiple methods.
    
    Args:
    text (str): The text to parse JSON from.
    
    Returns:
    Optional[Union[Dict[str, Any], list]]: Parsed JSON object, list, or None if parsing fails.
    """
    logger.debug(f"Attempting to parse JSON from: {text}")

    # Method 1: Direct JSON parsing
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        logger.debug(f"Initial JSON parse failed: {e}")

    # Method 2: Regex-based JSON-like structure extraction
    def extract_json_objects(text):
        obj_regex = r'\{(?:[^{}]|\{(?:[^{}]|\{[^{}]*\})*\})*\}'
        return re.findall(obj_regex, text)

    json_objects = extract_json_objects(text)
    for obj in json_objects:
        try:
            return json.loads(obj)
        except json.JSONDecodeError:
            continue

    # Method 3: Line-by-line parsing
    lines = text.split('\n')
    accumulated = ""
    for line in lines:
        accumulated += line
        try:
            return json.loads(accumulated)
        except json.JSONDecodeError:
            continue

    # Method 4: Manual key-value extraction
    def extract_key_values(text):
        kv_regex = r'"(\w+)"\s*:\s*(' \
                   r'"(?:\\.|[^"\\])*"' \
                   r'|\d+' \
                   r'|true|false|null' \
                   r'|\[(?:[^[\]]|\[(?:[^[\]]|\[[^[\]]*\])*\])*\]' \
                   r'|\{(?:[^{}]|\{(?:[^{}]|\{[^{}]*\})*\})*\}' \
                   r')'
        return re.findall(kv_regex, text, re.DOTALL)

    manual_parse = {}
    for key, value in extract_key_values(text):
        try:
            parsed_value = json.loads(value)
        except json.JSONDecodeError:
            parsed_value = value.strip('"')
        manual_parse[key] = parsed_value

    if manual_parse:
        logger.info(f"Manually parsed JSON: {manual_parse}")
        return manual_parse

    # Method 5: Attempt to parse as a JSON array
    array_regex = r'\[(.*)\]'
    array_match = re.search(array_regex, text, re.DOTALL)
    if array_match:
        try:
            return json.loads(f"[{array_match.group(1)}]")
        except json.JSONDecodeError:
            pass

    logger.warning("Failed to parse any JSON-like structure")
    return None

def safe_json_parser(func):
    """
    Decorator to safely parse JSON output from a function.
    
    Args:
    func: The function to decorate.
    
    Returns:
    A wrapper function that safely parses JSON.
    """
    async def wrapper(*args, **kwargs):
        result = await func(*args, **kwargs)
        if isinstance(result, str):
            parsed = parse_json_safely(result)
            if parsed is not None:
                return parsed
            else:
                logger.warning(f"Failed to parse JSON from function {func.__name__}")
                return result  # Return original string if parsing fails
        return result
    return wrapper